In [11]:
def elves():
    with open("day1.txt") as f:
        data=[line.strip() for line in f.readlines()]
        def chunks():
            p = 0
            for d in data:
                if d=='':
                    yield p
                    p = 0
                else:
                    p += int(d)
        return chunks()
day1a = max(elves())

In [12]:
day1b = sum(sorted(elves(), reverse=True)[0:3])

In [13]:
day1a, day1b

(68923, 200044)

In [34]:
def day2b():
    with open("day2.txt") as f:
        guide=[line.strip() for line in f.readlines()]
        guide = list(map(lambda x: tuple(x.split(" ")), guide))
    
    win = {'A':'Y','B':'Z','C':'X'}
    lose = {'A':'Z','B':'X','C':'Y'}
    draw = {'A':'X','B':'Y','C':'Z'}
    scoring = {'X':1,'Y':2,'Z':3}

    score = lambda opp, you: scoring[you] + (6 if win[opp] == you else 3 if draw[opp] == you else 0)
    move = {'X':lose,'Y':draw,'Z':win}
    
    total_basic = sum([score(opp,you) for opp,you in guide])
    total_advanced = sum([score(opp,(move[you])[opp]) for opp,you in guide])

    return total_basic, total_advanced
day2b()

(11906, 11186)

In [64]:
def day3():
    with open("day3.txt") as f:
        rucksacks = [line.strip() for line in f.readlines()]
        make_set = lambda x: set(list(x))
        rucksacks = list(map(lambda x: (make_set(x[:len(x)//2]),make_set(x[len(x)//2:])), rucksacks))
    priority = lambda p: ord(p) - ord('a') + 1 if p.islower() else ord(p) - ord('A') + 27

    s = 0
    for l,r in rucksacks:
        common = l.intersection(r)
        s += priority(min(common))
    print(s)
    
    s = 0
    groups = [[rucksacks[i],rucksacks[i+1],rucksacks[i+2]] for i in range(0,len(rucksacks),3)]
    for group in groups:
        m = None
        for l,r in group:
            m = l.union(r) if m is None else m.intersection(l.union(r))
        s += priority(min(m))
    print(s)

day3()

7746
2604


In [73]:
def day4():
    with open("day4.txt") as f:
        cfg = [line.strip() for line in f.readlines()]
        inc_range = lambda x,y: range(int(x),int(y)+1)
        mk_range = lambda x: inc_range(*x.split('-'))
        cfg = list(map(lambda c: list(map(set,map(mk_range,c.split(",")))), cfg))
        
    def overlaps(c):
        left,right = tuple(c)
        return len(left.intersection(right)) > 0
    
    def contains(c):
        left,right = tuple(c)
        return len(left.intersection(right)) == min(len(left),len(right))
    
    a = sum(map(contains, cfg))
    b = sum(map(overlaps, cfg))

    print(a,b)
day4()

490 921


In [113]:
import re
def day5():
    with open("day5.txt") as f:
        lines = map(lambda l: l.strip(), f.readlines())
        crates = [[]] * 9
        crates_b = [[]] * 9
        for l in lines:
            for i,s in enumerate(l[1::4]):
                if s.isupper():
                    crates[i] = [s] + crates[i]
                    crates_b[i] = [s] + crates_b[i]
            if l == "":
                break
        pattern = re.compile("move (.*) from (.*) to (.*)")
        for l in lines:
            n, fr, to = map(int, pattern.search(l).groups())
            for _ in range(n):
                c = crates[fr-1].pop()
                crates[to-1].append(c)
            crates_b[to-1] = crates_b[to-1] + crates_b[fr-1][-n:]
            del crates_b[fr-1][-n:]
        print("".join([c[-1] for c in crates]))
        print("".join([c[-1] for c in crates_b]))

day5()

SBPQRSCDF
RGLVRCQSB


In [138]:
def day6(marker_length = 4):
    with open("day6.txt") as f:
        l = list(f.readline())
    for i in range(len(l)-marker_length):
        if len(set(l[i:i+marker_length])) == marker_length:
            print(i+marker_length)
            break
day6()
day6(14)

1282
3513


In [193]:
def day7():
    with open("day7.txt") as f:
        lines = map(lambda l: l.rstrip(), f.readlines())
    root = tree = {}
    last_tree = [root]
    for line in lines:
        if line[0] == "$":
            cmd = line[2:].split(" ")
            if cmd[0] == "cd":
                d = cmd[1]
                if d == "..":
                    tree = last_tree.pop()
                else:
                    last_tree.append(tree)
                    if d not in tree:
                        tree[d] = {}
                    tree = tree[d]
        else:
            info, file = line.split(" ")
            if info == "dir":
                tree[file] = {}
            else:
                tree[file] = int(info)
                
    def file_size(tree):
        total = 0
        for k,v in tree.items():
            if type(v) == int:
                total += v
            else:
                total += file_size(v)
        return total
    
    def small_dir_size(tree):
        total = 0
        for k,v in tree.items():
            if type(v) is not int:
                s = file_size(v)
                if s <= 100000:
                    total += s
                total += small_dir_size(v)
        return total
    
    print(small_dir_size(root))
    root_size = file_size(root)
    to_free = root_size - 4e7
    
    smallest = [None]
    def collect_smallest(tree, sm):
        for k,v in tree.items():
            if type(v) is not int:
                size = file_size(v)
                if size >= to_free and (sm[0] is None or sm[0] > (file_size(v)-to_free)):
                    sm[0] = file_size(v)-to_free
                collect_smallest(v, sm)
                
    collect_smallest(root, smallest)
    print(smallest[0]+to_free)
day7()
        

1743217
8319096.0


In [274]:
from collections import defaultdict
def day8():
    with open("day8.txt") as f:
        t = list(map(lambda l: list(l.rstrip()), f.readlines()))
    
    h = len(t)
    w = len(t[0])
    v = {}
    s = {}
    
    for y in range(h):
        for x in range(w):
            tree = t[y][x]
            los_x = [reversed(range(0,x)),range(x+1,w)]
            los_y = [reversed(range(0,y)),range(y+1,h)]
            scores = []
            
            tree_los = []
            
            for los in los_x:
                tree_los.append(list(map(lambda T: T < tree, [t[y][x] for x in los])))
            for los in los_y:
                tree_los.append(list(map(lambda T: T < tree, [t[y][x] for y in los])))
                
            for trees in tree_los:
                if all(trees):
                    v[(x,y)] = True
                    score = len(list(trees))
                else:
                    for idx,T in enumerate(trees):
                        if not T:
                            score = idx+1
                            break
                    
                scores.append(score)

            s[(x,y)] = scores[0]*scores[1]*scores[2]*scores[3]
            
    print(len(v.keys()))
    print(max(s.values()))
            
day8()
            

1662
537600


In [365]:
from PIL import Image, ImageDraw

def day9(N = 2):
    with open("day9.txt") as f:
        move = lambda move: {'R': (1,0), 'L': (-1,0), 'D': (0,1), 'U': (0,-1)}[move]
        tuples = list(map(lambda l: l.rstrip().split(" "), f.readlines()))
        moves = list(map(lambda t: (move(t[0]), int(t[1])), tuples))
    
    frames = []
    nodes = [[0,0] for i in range(N)]
    visited = set([(0,0)])
    for move in moves:
        for i in range(move[1]):
            head = nodes[0]
            head[0] += move[0][0]
            head[1] += move[0][1]

            for r in range(0,N-1):
                head = nodes[r]
                tail = nodes[r+1]
                
                if (tail[0] == head[0] and abs(tail[1]-head[1]) == 2):
                    tail[1]=head[1]+(1 if tail[1]>head[1] else -1)
                if (tail[1] == head[1] and abs(tail[0]-head[0]) == 2):
                    tail[0]=head[0]+(1 if tail[0]>head[0] else -1)
                if (tail[0] != head[0] and tail[1] != head[1] and not \
                    (abs(tail[0]-head[0]) == 1 and abs(tail[1]-head[1]) == 1)):
                    tail[0] += (1 if tail[0]<head[0] else -1)
                    tail[1] += (1 if tail[1]<head[1] else -1)
            visited.add(tuple(nodes[-1]))
            new_frame = Image.new("RGB", (300,300))
            draw = ImageDraw.Draw(new_frame)
            coords = list(map(lambda n: (n[0]*4+200, n[1]*4+100),nodes))
            draw.line(coords, fill="red", width=4)
            
        frames.append(new_frame)
    frames[0].save('rope.gif', format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)
    
    print(max(visited))
    print(min(visited))

    print(len(visited))
    
#day9()
day9(36)

(36, 25)
(-278, 119)
843


In [390]:
def day10():
    with open("day10.txt") as f:
        cmds = list(map(lambda l: l.rstrip().split(" "), f.readlines()))
        
    cycle = 0
    x = 1
    samples = []
    crt = [[] for _ in range(6)]
    
    def signal(cycle, samples, x):
        row = (cycle-1)//40
        char = "#" if abs(len(crt[row])-x) <= 1 else "."
        crt[row].append(char)
        if (cycle-20)%40 == 0:
            samples.append(x * cycle)
        
    for cmd in cmds:
        if cmd[0] == "noop":
            cycle += 1
            signal(cycle, samples, x)
            
        if cmd[0] == "addx":
            cycle += 1
            signal(cycle, samples, x)
            cycle += 1
            signal(cycle, samples, x)
            x += int(cmd[1])
        
    print(sum(samples))
    for row in crt:
        print("".join(row))
day10()

13860
###..####.#..#.####..##....##..##..###..
#..#....#.#..#.#....#..#....#.#..#.#..#.
#..#...#..####.###..#.......#.#....###..
###...#...#..#.#....#.##....#.#....#..#.
#.#..#....#..#.#....#..#.#..#.#..#.#..#.
#..#.####.#..#.#.....###..##...##..###..
